In [1]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt



import glob
from tqdm.notebook import tqdm


from nltk import sent_tokenize, word_tokenize, regexp_tokenize
from nltk.corpus import stopwords

import nltk
nltk.download('punkt')

from collections import Counter

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\upadh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
dis = pd.read_csv("../data/text/discharge.csv", nrows=100)

In [3]:
dis.head()

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00,\nName: ___ Unit No: _...
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00,\nName: ___ Unit No: _...
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25 00:00:00,2180-07-25 21:42:00,\nName: ___ Unit No: _...
3,10000032-DS-24,10000032,25742920,DS,24,2180-08-07 00:00:00,2180-08-10 05:43:00,\nName: ___ Unit No: _...
4,10000084-DS-17,10000084,23052089,DS,17,2160-11-25 00:00:00,2160-11-25 15:09:00,\nName: ___ Unit No: __...


In [4]:
dis.loc[0,'text']

' \nName:  ___                     Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___\n \nChief Complaint:\nWorsening ABD distension and pain \n \nMajor Surgical or Invasive Procedure:\nParacentesis\n\n \nHistory of Present Illness:\n___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, \nbioplar, PTSD, presented from OSH ED with worsening abd \ndistension over past week.  \nPt reports self-discontinuing lasix and spirnolactone ___ weeks \nago, because she feels like "they don\'t do anything" and that \nshe "doesn\'t want to put more chemicals in her." She does not \nfollow Na-restricted diets. In the past week, she notes that she \nhas been having worsening abd distension and discomfort. She \ndenies ___ edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, \ndysuria. She had food poisoning a week ago from eatin

In [5]:
seed = 123
for statement in dis.loc[0:3, 'text'].sample(3, random_state=seed):
    print(statement)
    print('-------------------------------------------------------------------------')

 
Name:  ___                     Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: MEDICINE
 
Allergies: 
Percocet / Vicodin
 
Attending: ___.
 
Chief Complaint:
Abdominal pain
 
Major Surgical or Invasive Procedure:
___ Paracentesis

 
History of Present Illness:
___ w/ HIV on HAART, COPD on 3L home O2 (though sat'ing fine on 
RA on admission), HCV cirrhosis c/b ascites requiring biweekly 
therapeutic paracenteses, hepatic encephalopathy; not on 
transplant list ___ comorbidities) p/w worsening girth and abd 
pain. She'd been having pain from her ascites, and felt overdue 
for a paracentesis. She last had paracentesis on the ___. 
She reported that she began feeling worsening abdominal pain 
(constant, epigastric and radiates to back, not a/w food) on 
___, which increased in severity throughout the course of the 
day. She was brought to the ED by her son because of this pain. 
She had no confusion and was al

In [50]:
tt = dis.loc[0:3,'text']
type(tt[0])
tt

0     \nName:  ___                     Unit No:   _...
1     \nName:  ___                     Unit No:   _...
2     \nName:  ___                     Unit No:   _...
3     \nName:  ___                     Unit No:   _...
Name: text, dtype: object

In [36]:
text_start = re.search('Major Surgical or Invasive Procedure:\n.+', dis.loc[0,'text']).end()
text_end = re.search('Past Medical History:\n.+', dis.loc[0,'text']).start()

chief_complaint = dis.loc[0,'text'][text_start:text_end]

In [37]:
chief_complaint

'\n\n \nHistory of Present Illness:\n___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, \nbioplar, PTSD, presented from OSH ED with worsening abd \ndistension over past week.  \nPt reports self-discontinuing lasix and spirnolactone ___ weeks \nago, because she feels like "they don\'t do anything" and that \nshe "doesn\'t want to put more chemicals in her." She does not \nfollow Na-restricted diets. In the past week, she notes that she \nhas been having worsening abd distension and discomfort. She \ndenies ___ edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, \ndysuria. She had food poisoning a week ago from eating stale \ncake (n/v 20 min after food ingestion), which resolved the same \nday. She denies other recent illness or sick contacts. She notes \nthat she has been noticing gum bleeding while brushing her teeth \nin recent weeks. she denies easy bruising, melena, BRBPR, \nhemetesis, hemoptysis, or hematuria.  \nBecause of her abd pain, she went to OSH ED and was transf

In [40]:
dis.loc[0,'text']

' \nName:  ___                     Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___\n \nChief Complaint:\nWorsening ABD distension and pain \n \nMajor Surgical or Invasive Procedure:\nParacentesis\n\n \nHistory of Present Illness:\n___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, \nbioplar, PTSD, presented from OSH ED with worsening abd \ndistension over past week.  \nPt reports self-discontinuing lasix and spirnolactone ___ weeks \nago, because she feels like "they don\'t do anything" and that \nshe "doesn\'t want to put more chemicals in her." She does not \nfollow Na-restricted diets. In the past week, she notes that she \nhas been having worsening abd distension and discomfort. She \ndenies ___ edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, \ndysuria. She had food poisoning a week ago from eatin

In [ ]:
#if need allergies work seprately

In [43]:
list_subheading = re.findall('\n(.+):\n', dis.loc[0,'text'])

In [55]:
list_subheading = list_subheading+['$']

In [48]:
text_start = re.search(list_subheading[0], dis.loc[0,'text']).end()
text_end = re.search(list_subheading[1], dis.loc[0,'text']).start()

chief_complaint = dis.loc[0,'text'][text_start:text_end]

In [49]:
chief_complaint

':\nWorsening ABD distension and pain \n \n'

In [56]:
list_subheading

['Chief Complaint',
 'Major Surgical or Invasive Procedure',
 'History of Present Illness',
 'Past Medical History',
 'Social History',
 'Family History',
 'Physical Exam',
 'Discharge',
 'Pertinent Results',
 'Brief Hospital Course',
 'Medications on Admission',
 'Discharge Medications',
 'Discharge Disposition',
 'Discharge Diagnosis',
 'Discharge Condition',
 'Discharge Instructions',
 'Followup Instructions',
 '$']

In [70]:
# text_start = re.search(list_subheading[0], dis.loc[0,'text']).end()
# text_end = re.search(list_subheading[1], dis.loc[0,'text']).start()

# chief_complaint = dis.loc[0,'text'][text_start:text_end]
#parsed text into dic becoz we may have different subheadings

dic_list = []
for j in range (10):
    
    dic = {}
    # j = 5
    list_subheading = re.findall('\n(.+):\n', dis.loc[j,'text'])
    list_subheading = list_subheading+['$']


    for i in range (len(list_subheading)-1):
        text_start = re.search(list_subheading[i] + ":\n", dis.loc[j,'text']).end()
        if i < (len(list_subheading)-2):
            text_end = re.search(list_subheading[i+1] + ":\n", dis.loc[j,'text']).start()
        else:
            text_end = re.search(list_subheading[i+1], dis.loc[j,'text']).start() 
        text_parsed = dis.loc[j,'text'][text_start:text_end]

        dic[list_subheading[i]]=text_parsed
    dic_list.append(dic) 
#     print(list_subheading[i], text_parsed)
#     print("------------------------------")
# print(dic)
    
    

In [71]:
pd.DataFrame(dic_list)
#merge this on the index with the current df.

,Chief Complaint,Major Surgical or Invasive Procedure,History of Present Illness,Past Medical History,Social History,Family History,Physical Exam,Discharge,Pertinent Results,Brief Hospital Course,...,NECK X-ray ___,SECONDARY DIAGNOSIS,Left Lower extremity,ACTIVITY AND WEIGHT BEARING,MEDICATIONS,how to wean down,ANTICOAGULATION,# TRANISTIONAL ISSUES,DISCHARGE,CARDIAC MARKER TREND
0,Worsening ABD distension and pain \n \n,Paracentesis\n\n \n,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",1. HCV Cirrhosis \n2. No history of abnormal ...,___\n,"She a total of five siblings, but she is not ...",VS: 98.1 107/61 78 18 97RA \nGeneral: in NAD ...,\nPHYSICAL EXAMINATION: \nVS: 98 105/70 95\nG...,___ 10:25PM GLUCOSE-109* UREA N-25* CREAT-0....,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,abdominal fullness and discomfort\n \n,___ diagnostic paracentesis\n___ therapeutic p...,"___ with HIV on HAART, COPD, HCV cirrhosis com...",1. HCV Cirrhosis \n2. No history of abnormal ...,___\n,"She a total of five siblings, but she is not ...",ADMISSION PHYSICAL EXAM: \nVS: T98.1 105/57 7...,NaN,LABS ON ADMISSION:\n___ 04:10PM BLOOD ___ \n__...,"___ with HIV on HAART, HCV cirrhosis with asci...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,altered mental status\n \n,none\n \n,"Mrs. ___ is a ___ female with HIV on HAART, CO...",- HCV Cirrhosis: genotype 3a \n- HIV: on HAAR...,___\n,"She a total of five siblings, but she is not ...",ADMISSION PHYSICAL EXAM:\nVitals T: 98.7 BP: 8...,NaN,ADMISSION LABS:\n=================\n___ 06:39A...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Abdominal pain\n \n,___ Paracentesis\n\n \n,"___ w/ HIV on HAART, COPD on 3L home O2 (thoug...",- HCV Cirrhosis: genotype 3a \n- HIV: on HAAR...,___\n,"She a total of five siblings, but she is not ...",ADMISSION PHYSICAL EXAM\n=====================...,NaN,ADMISSION LABS\n==============\n___ 09:20PM BL...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Visual hallucinations\n \n,N/A\n\n \n,"___ male with ___ disease, dyslipidemia, and a...",___ disease\n___ Body Dementia \ndyslipidemia ...,___\n,"His mother died at age ___ of ""old age.""\nHis ...",ADMISSION PHYSICAL EXAM:\n====================...,NaN,ADMISSION LABS:\n===============\n___ 10:30PM ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,dysphagia \n \n,Upper endoscopy ___\n\n \n,___ w/ anxiety and several years of dysphagia...,- GERD \n - Hypercholesterolemia \n - Kidney...,___\n,+ HTN - father \n+ Dementia - father \n \n \n,=================\nADMISSION/DISCHARGE EXAM\n=...,NaN,=============\nADMISSION LABS\n=============\n...,Ms. ___ is a ___ with history of GERD who pres...,...,IMPRESSION: \n \nWithin the limitation of pl...,-GERD \n\n \n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Left hip pain \n \n,"Status post left CRPP ___, ___\n\n \n",REASON FOR CONSULT: Femur fracture\n\nHPI: ___...,- GERD \n - Hypercholesterolemia \n - Kidney...,___\n,+ HTN - father \n+ Dementia - father \n \n \n,General: Well-appearing female in no acute dis...,NaN,NaN,The patient presented to the emergency departm...,...,NaN,NaN,"- Skin intact\n- No deformity, edema, ecchymos...",- Weightbearing as tolerated left lower extrem...,1) Take Tylenol ___ every 6 hours around th...,"Take 1 tablet every 3 hours as needed x 1 day,...",- Please take Lovenox daily for 4 weeks\n \n,NaN,NaN,NaN
7,Right flank bruising and pain s/p fall\n \n,None\n\n \n,Mr. ___ is a ___ with history of factor VIII d...,-Factor VIII deficiency (mild) \n\n \n,___\n,The patient's mother had tendency to bleed. \...,ON ADMISSION\nVS:98 120/40 64 20 100RA \nGENE...,NaN,"ADMISSION, DISCHARGE, PERTINENT LABS:\n\n___ 0...",Mr. ___ is a ___ with history of mild FVIII de...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- PCP ___ ___\n- CBC ___ - ___ at ___\n- Cauti...,NaN,NaN
8,renal mass\n \n,right laparascopic radical nephrectomy- Dr. __...,___ y/o healthy female with incidental finding...,PMH: nonspecific right axis deviation \n\nPSH...